# Fitting AUSF

The model requires the fitting of pig population dynamics and ASF disease dynamics. We first fit the population to ensure that the population returns to the carrying capacity

### Load Modules

In [ ]:
using GpABC, Distances, Plots, DifferentialEquations, LinearAlgebra, SparseArrays, Distributions, Random

In [ ]:
#Loading the functions to build the input and run the models
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");

In [ ]:
input_path = "/home/callum/ASF/Inputs/"; #path to model data

### Tau-leaping model

In [ ]:
input = ASF_Inputs.Model_Data(input_path, fitting_rewire = 1.3, verbose = true); #loading the model input

In [ ]:
const nt = input.Parameters.Populations.cum_sum[end] #total number of groups and/or farms
const nc = 5 #number of classes (SEIRC)
const eqs = 11 #number of processes

#Matrix of all the transitions between classes for Gillespie model
const dc = sparse(zeros(nt*nc,nt*eqs))

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [ ]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
    nothing
end

In [ ]:
# function that breaks up results into easy to analyse form
function run_analysis(sol)
   
        data = reduce(vcat,transpose.(sol.u))
        data[data .< 0 ] .= 0
   
        s_d = data[:,1:5:end]
        e_d = data[:,2:5:end]
        i_d = data[:,3:5:end]
        r_d = data[:,4:5:end]
        c_d = data[:,5:5:end]
 
        disease_total = e_d + i_d + c_d #classes with disease,
        disease_alive = e_d + i_d
 
        disease_free = s_d + r_d #classes without disease,
 
        disease_sum = sum(disease_total,dims=2)
        disease_alive_sum =  sum(disease_alive,dims=2)
        disease_free_sum = sum(disease_free,dims=2)
        population_sum = disease_alive_sum + disease_free_sum;
   
        return disease_sum, disease_alive_sum, disease_free_sum, population_sum
    end

## Fitting population in tau-leaping model

There is population level drift, therefore we fit the population (by adjusting deaths) to make the population stabalise at carrying capacity. This effect occurs due to the small population size (often, given the equations, a death is more preferable than a birth).

In [ ]:
AUSF_pop = function(params,Tspan, U0)
    #function that runs model for fitting

    rj = RegularJump(ASF_Model.asf_model_pop, regular_c, eqs*nt)

    prob = DiscreteProblem(U0,Tspan,params)

    jump_prob = JumpProblem(prob,Direct(),rj)

    ensemble_prob_m = EnsembleProblem(jump_prob)

    obs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);
    
    #simply averaging over our mutliple runs!
    output_final = Matrix{Float64}(undef,(1,n_years*365+1,))
    output_final .= 0
    for i in 1:n_runs
        output_final += sum(hcat(obs[1].u...), dims=1)/n_runs
    end
    
    return output_final
end
    

function simulator_function_pop(var_params)
    
    #function that builds the input and runs fitting!
    input = ASF_Inputs.Model_Data(input_path); #input!

    U0 = copy(input.U0); #initial pop
    U0[1:5:end] += (U0[2:5:end] +U0[3:5:end]) #setting init pop to zero
    U0[2:5:end] .= 0 #removing infected populations
    U0[3:5:end] .= 0
    
    params = ASF_Model.convert(input.Parameters)
    
    params[15] = var_params[1] #The death modifier, to reduce population drift 
    
    Time = (0.0,n_years*365.0)
    #now we run simulation!
    
    AUSF_pop(params, Time, U0)
end


In [ ]:
#Some key parameters for fitting!

input_path = "/home/callum/ASF/Inputs/"; #path to model data
n_years = 5 #how many years each sim will run for
n_runs = 3; #how many runs for each param


### Building Obersevations!

In [ ]:
function density_carrying!(du,u,p,t)
    #ODE to calculate the "real" observations
    S = u[1]
    k, bw, σ, K, μ_p = p
  
    du[1] =  k*exp(-bw*cos(pi*(t)/365)^2).*(σ .* S .+ ((1-σ)) .* sqrt.(S.*K))-S.*μ_p.*(σ .+ ((1-σ)).*sqrt.(S./K))
    
end

In [ ]:
input = ASF_Inputs.Model_Data(input_path);
U0_ode = [sum(input.U0)];
p_ode = [input.Parameters.k[1], input.Parameters.bw[1],input.Parameters.σ[1],sum(input.Parameters.K),input.Parameters.μ_p[1]];
tspan_ode = (0.0,1000*365.0);

#solving ODE
prob_ode = ODEProblem(density_carrying!, U0_ode, (0.0,n_years*365.0), p_ode)
sol_ode = solve(prob_ode, saveat = 1,reltol=1e-8)
obs = hcat(sol_ode.u...);

In [ ]:
plot(obs', xlabel = "Time (days)", ylabel = "Population", label="")


In [ ]:
#Our params for fitting
n_particles = 100 #number of acceptances we want
devi = 0.975 #how close (from 0-1 with 1 being identical) do we want output to be to observed, used for threshold
threshold = euclidean(obs,0.95*obs) #want to be with 95% of "real observation" to be accepted;
priors = [Uniform(0.9,1.0)]

In [ ]:
sim_result = SimulatedABCRejection(obs, simulator_function_pop, priors, threshold, n_particles; max_iter=10000, write_progress=false)#,

In [ ]:
plot(sim_result)

In [ ]:
data = sim_result.population
vars = ["g"]

wanted = data[:,1]
println(vars[1])
println("Median: ", round(median(wanted),digits=4))
println("Mean: ", round(mean(wanted), digits=4))
    


## Fitting  tau-laping model ASF

Here we are fitting the diease dynamics to previously observed data from outbreaks in the Baltic States and Poland

In [ ]:
#Important means

m_t = Normal(180,36.48)
p_d = Normal(75,6.08)
e_p = Normal(1.5,0.304)

In [ ]:
function AUSF_tau(dis_params,Tspan,U0)
    rj = RegularJump(ASF_Model.asf_model_one, regular_c, eqs*nt)
    prob = DiscreteProblem(U0,Tspan,dis_params)
    jump_prob = JumpProblem(prob,Direct(),rj)
    ensemble_prob = EnsembleProblem(jump_prob)
    model_outs = solve(ensemble_prob, SimpleTauLeaping(), trajectories= 3,EnsembleThreads(),dt=1)
    
    res = eval_out(model_outs)
    
end
    
function simulator_function_d(var_params)
    start_day = 180
    
    Tspan = (start_day,365*n_years+start_day)
    
    input = ASF_Inputs.Model_Data(input_path,var_params[4]);
    
    params = ASF_Model.convert(input.Parameters)
    
    U0 = input.U0

    Burn_U0 =  copy(U0)
    Burn_U0[2:5:end] .= 0
    Burn_U0[3:5:end] .= 0
    rj_burn = RegularJump(ASF_Model.asf_model_pop, regular_c, eqs*nt)
    prob_burn = DiscreteProblem(Burn_U0,(0.0,275),params)
    jump_prob_burn = JumpProblem(prob_burn, Direct(), rj_burn)
    sol_burn = solve(jump_prob_burn, SimpleTauLeaping(),dt=1);
    
    U_burn = copy(sol_burn[params[18]+start_day]); #population at start date
    rr = rand(1:nt) #seeding diease in starting pop
    ra = rr -1
    if U_burn[ra*5+1] > 1
        U_burn[ra*5+1] = 0
        U_burn[ra*5+2] = 3
        U_burn[ra*5+3] = 2
    else
        U_burn[ra*5+1] = 0
        U_burn[ra*5+3] = 1
    end

    netw = params[3][:,rr] #related populations
    cons = findall(>(0), netw)
    
    if length(cons) <= 4
        wanted = cons
    else
        wanted=shuffle(cons)[1:4] #seeding in 4 other pops so 5 in total!
    end 
    
    for i in cons
        i1 = i -1
        if U_burn[i1*5+1] > 1
            U_burn[i1*5+1] = 0
            U_burn[i1*5+2] = 3
            U_burn[i1*5+3] = 2
        else
            U_burn[i1*5+1] = 0
            U_burn[i1*5+3] = 1
        end

    end
    #beta
    params[1] .= var_params[1] #intra
    params[2][params[2] .!= 0 ] .= var_params[2]/n_con #inter
    
    #corpse 
    params[8] = var_params[3] #corpse infection modifier
    AUSF_tau(params, Tspan, U_burn)
    
    
end


function eval_out(data) 

    output_final = Matrix{Float64}(undef,(3,1))
    s = 0
    ep = 0
    mt = 0
    pd = 0
    detection_p = 0.05
    pop_K = 5000
    
    starting_p = detection_p*pop_K
    for i in 1:3
        d, da,f,p = run_analysis(data[i])

        if d[end] > 0
            
            ep += 100*mean(da[3*365:end])/mean(p[3*365:end])
            pd += 100*(1-mean(p[3*365:end])/pop_K)
            
            max_d = findmax(d)[2][1]
            
            if maximum(d) <= starting_p
                take_off_time = 0
            else
                take_off_time = findfirst(>(starting_p), d)[1]
            end
            
            mt += max_d-take_off_time
            s += 1

        end

    end
    
    if s >= 2
        output_final[1] = pdf.(e_p,ep/s)/pdf.(e_p,mean(e_p))
        output_final[2] = pdf.(p_d,pd/s)/pdf.(p_d,mean(p_d))
        output_final[3] = pdf.(m_t,mt/s)/pdf.(m_t,mean(m_t))

        
    else
        output_final[1] = 0
        output_final[2] = 0
        output_final[3] = 0
       
    end
   
    
    return output_final
    
end

function observed_params()
   
    obs =  Matrix{Float64}(undef,(3,1))
    
    obs[1] = 1
    obs[2] = 1
    obs[3] = 1
    
    return obs
end

In [ ]:
thresh_shed = [1.2841903149896468,0.7197669138598748, 0.5165660265550855]

In [ ]:
input_path = "/home/callum/ASF/Inputs/"; #path to model data
n_years = 10 #how many years each sim will run for
n_runs = 3 #how many runs for each param
n_con = 6  # average number of connections each group has
obs = observed_params() #our observed (just array of ones, we normalise output to be relative to this)
priors = [Uniform(0.1, 1.0), Uniform(0.01, 0.1), Uniform(0, 1.0), Uniform(0,0.5)];

In [ ]:
n_particles = 250;

In [ ]:
sim_abcsmc_res = SimulatedABCSMC(obs,simulator_function_d,priors,thresh_shed,n_particles; write_progress=true)

In [ ]:
ds = 3
data = sim_abcsmc_res.population[3]
vars = ["β_intra", "β_inter", "ω", "p"]
for i in 1:4
    wanted = data[:,i]
    println(vars[i])
    println("Median: ", round(median(wanted),digits=4))
    println("Mean: ", round(mean(wanted), digits=4))
    
    println()
end

## Fitting ODE Model
Now we fit ODE Model, needs to be fit seperately as no population drift and no groups!
Should be alot faster than tau-leaping fitting!

In [ ]:
function AUSF_ode(param_ode,Tspan,U0)
    
    prob_ode = ODEProblem(ASF_Model.asf_model_ode, U0_ode, (0.0,n_years*365.0), param_ode)
    sol_ode = solve(prob_ode, saveat = 1,reltol=1e-8)
    
    res = eval_out(sol_ode)
    
end
    
function simulator_function_d(var_params)
    start_day = 180
    
    Tspan = (start_day,365*n_years+start_day)
    
    input = ASF_Inputs.Model_Data(input_path,var_params[4]);
    
    params = ASF_Model.convert(input.Parameters)
    
    U0 = input.U0

    Burn_U0 =  copy(U0)
    Burn_U0[2:5:end] .= 0
    Burn_U0[3:5:end] .= 0
    rj_burn = RegularJump(ASF_Model.asf_model_pop, regular_c, eqs*nt)
    prob_burn = DiscreteProblem(Burn_U0,(0.0,275),params)
    jump_prob_burn = JumpProblem(prob_burn, Direct(), rj_burn)
    sol_burn = solve(jump_prob_burn, SimpleTauLeaping(),dt=1);
    
    U_burn = copy(sol_burn[params[18]+start_day]); #population at start date
    rr = rand(1:nt) #seeding diease in starting pop
    ra = rr -1
    if U_burn[ra*5+1] > 1
        U_burn[ra*5+1] = 0
        U_burn[ra*5+2] = 3
        U_burn[ra*5+3] = 2
    else
        U_burn[ra*5+1] = 0
        U_burn[ra*5+3] = 1
    end

    netw = params[3][:,rr] #related populations
    cons = findall(>(0), netw)
    
    if length(cons) <= 4
        wanted = cons
    else
        wanted=shuffle(cons)[1:4] #seeding in 4 other pops so 5 in total!
    end 
    
    for i in cons
        i1 = i -1
        if U_burn[i1*5+1] > 1
            U_burn[i1*5+1] = 0
            U_burn[i1*5+2] = 3
            U_burn[i1*5+3] = 2
        else
            U_burn[i1*5+1] = 0
            U_burn[i1*5+3] = 1
        end

    end
    #beta
    params[1] .= var_params[1] #intra
    params[2][params[2] .!= 0 ] .= var_params[2]/n_con #inter
    
    #corpse 
    params[8] = var_params[3] #corpse infection modifier
    AUSF_d(params, Tspan, U_burn)
    
    
end
